In [1]:
# Install required packages
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers datasets gradio accelerate
!pip install -q bitsandbytes  # For memory-efficient training

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.2 MB/s eta 0:00:00


In [2]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from datasets import Dataset, load_dataset
import pandas as pd
import gradio as gr
from transformers import set_seed

# Set seed for reproducibility
set_seed(7)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
# Load smaller, faster model
model_name = "sshleifer/distilbart-cnn-12-6"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Load dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:5000]")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
def preprocess_data(examples):
    inputs = tokenizer(
        examples["article"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )
    targets = tokenizer(
        examples["highlights"],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )
    targets["input_ids"][targets["input_ids"] == tokenizer.pad_token_id] = -100
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(
    preprocess_data,
    batched=True,
    remove_columns=["article", "highlights", "id"]
)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [5]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./optimized_summarizer",
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=400,
    learning_rate=3e-5,  # Slightly lower LR for stability
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=50,
    report_to=None,
    dataloader_pin_memory=False,
    gradient_checkpointing=True,
    warmup_steps=100,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset.select(range(500)),
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)
)


/tmp/ipython-input-134167453.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [6]:
trainer.train()
trainer.save_model("./optimized_summarizer_final")
tokenizer.save_pretrained("./optimized_summarizer_final")


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: majedabbadi08 (majedabbadi08-no-employer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, mcp] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss,Validation Loss
200,1.877800,1.735942
400,1.822200,1.630968
600,1.885700,1.556089
800,1.834800,1.517926
1000,1.708200,1.452024
1200,1.812200,1.408103
1400,1.700900,1.323477
1600,1.706400,1.279563
1800,1.751900,1.230948
2000,1.771400,1.194524


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./optimized_summarizer_final/tokenizer_config.json',
 './optimized_summarizer_final/special_tokens_map.json',
 './optimized_summarizer_final/vocab.json',
 './optimized_summarizer_final/merges.txt',
 './optimized_summarizer_final/added_tokens.json',
 './optimized_summarizer_final/tokenizer.json')

In [7]:
# Enhanced inference pipeline with better parameters
summarizer = pipeline(
    "summarization",
    model="./optimized_summarizer_final",
    tokenizer=tokenizer,
    device=0 if device.type == 'cuda' else -1,
    max_length=128,
    min_length=20,
    do_sample=True,  # Enable sampling for more diverse output
    top_p=0.9,
    temperature=0.7,
    repetition_penalty=1.2
)

def summarize_news(text):
    if len(text.strip()) < 50:
        return "Please enter a longer article (at least 50 characters)"

    # Handle long texts by chunking
    if len(text) > 1024:
        text = text[:1024]  # Truncate very long texts

    try:
        result = summarizer(
            text,
            max_length=128,
            min_length=20,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.2
        )
        return result[0]['summary_text']
    except Exception as e:
        return f"Error processing text: {str(e)}"


/usr/local/lib/python3.12/dist-packages/transformers/models/bart/configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
Device set to use cuda:0


In [9]:
with gr.Blocks(theme=gr.themes.Default(primary_hue="blue", secondary_hue="slate"), title="News Summarizer") as demo:
    gr.Markdown(
        """
        # 📰 AI News Summarizer
        *Transform lengthy articles into concise summaries*
        """
    )

    with gr.Row():
        with gr.Column():
            article_input = gr.TextArea(
                label="Enter Article",
                placeholder="Paste your news article or document here...",
                lines=12,
                value="Google Cloud service integration: Provides AI assistance directly within services like Firebase (app error analysis, performance insights), Colab Enterprise (Python code generation), BigQuery (natural language to SQL, query optimization), Cloud Run, and Apigee"
            )
            submit_btn = gr.Button("Generate Summary", variant="primary", size="lg")

        with gr.Column():
            summary_output = gr.TextArea(
                label="Summary",
                placeholder="Summary will appear here...",
                lines=12,
                interactive=False
            )

    gr.Examples(
        examples=[
            ["Google Cloud service integration: Provides AI assistance directly within services like Firebase (app error analysis, performance insights), Colab Enterprise (Python code generation), BigQuery (natural language to SQL, query optimization), Cloud Run, and Apigee"]
        ],
        inputs=article_input,
        outputs=summary_output,
        label="Example Articles"
    )

    gr.Markdown(
        """
        > **Note:** Works best with news-style content. For technical documents, results may vary.
        """
    )

    submit_btn.click(
        fn=summarize_news,
        inputs=article_input,
        outputs=summary_output
    )

demo.launch(share=True)

/tmp/ipython-input-2683159447.py:1: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Default(primary_hue="blue", secondary_hue="slate"), title="News Summarizer") as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e3ed7d0e1b4d71626f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
